# Language Modeling & Sentiment Analysis of IMDB movie reviews

We will be looking at IMDB movie reviews.  We want to determine if a review is negative or positive, based on the text.  In order to do this, we will be using **transfer learning**.

Transfer learning has been widely used with great success in computer vision for several years, but only in the last year or so has it been successfully applied to NLP (beginning with ULMFit, which we will use here, which was built upon by BERT and GPT-2).

As Sebastian Ruder wrote in [The Gradient](https://thegradient.pub/) last summer, [NLP's ImageNet moment has arrived](https://thegradient.pub/nlp-imagenet/).

# Modelado de lenguaje y análisis de opinión de comentarios de películas IMDB

Vamos a ver comentarios de películas de IMDB. Queremos determinar si una revisión es negativa o positiva, según el texto. Para hacer esto, utilizaremos el aprendizaje de transferencia.

El aprendizaje por transferencia se ha utilizado ampliamente con gran éxito en visión por computadora durante varios años, pero solo en el último año más o menos se ha aplicado con éxito a NLP (comenzando con ULMFit, que usaremos aquí, que fue desarrollado por BERT y GPT -2).

Como Sebastian Ruder escribió en [The Gradient](https://thegradient.pub/) el verano pasado, [ha llegado el momento ImageNet de NLP](https://thegradient.pub/nlp-imagenet/).

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#%%bash
#pip install fastai

In [ ]:
from fastai import *
from fastai.text import *

In [ ]:
# import fastai.utils.collect_env
# fastai.utils.collect_env.show_install()

Note that language models can use a lot of GPU, so you may need to decrease batchsize here.

Ten en cuenta que los modelos de lenguaje pueden usar una gran cantidad de GPU, por lo que es posible que deba reducir el tamaño del lote (batch) aquí.

In [ ]:
# bs=48
bs=24
#bs=192

In [ ]:
#torch.cuda.set_device(0)

## Preparing the data (on a sample)

First let's download the dataset we are going to study. The [dataset](http://ai.stanford.edu/~amaas/data/sentiment/) has been curated by Andrew Maas et al. and contains a total of 100,000 reviews on IMDB. 25,000 of them are labelled as positive and negative for training, another 25,000 are labelled for testing (in both cases they are highly polarized). The remaning 50,000 is an additional unlabelled data (but we will find a use for it nonetheless).

We'll begin with a sample we've prepared for you, so that things run quickly before going over the full dataset.

## Preparación de los datos (en una muestra)

### Esta parte se encuentra comentada, ya que no será necesario bajar los datos: se usará una copia de ellos que ya se encuentra en la máquina virtual. 
Primero descarguemos el conjunto de datos que vamos a estudiar. El [conjunto de datos](http://ai.stanford.edu/~amaas/data/sentiment/) ha sido preparado por Andrew Maas et al. y contiene un total de 100,000 comentarios en IMDB. 25,000 de ellos están etiquetados como positivos y negativos para el entrenamiento, otros 25,000 están etiquetados para pruebas (en ambos casos están altamente polarizados). Los 50,000 restantes son datos adicionales no etiquetados (pero de todos modos lo usaremos).

Comenzaremos con una muestra que hemos preparado para ustedes, de modo que las cosas se ejecuten rápidamente antes de revisar el conjunto de datos completo.

In [ ]:
#path = untar_data(URLs.IMDB_SAMPLE)
#path.ls()

Enlace simbolico (symlink / acceso directo) de la carpeta con los datos en la ubicacion correspondiente. 

In [ ]:
!mkdir ~/.fastai
!mkdir ~/.fastai/data
!mkdir ~/.fastai/data/imdb_sample
!ln -s /data/home/admin101/.fastai/data/imdb_sample/t* ~/.fastai/data/imdb_sample/

In [ ]:
path = Config.data_path()/'imdb_sample'
path.mkdir(parents=True, exist_ok=True)
path.ls()

It contains one line per review, with the label ('negative' or 'positive'), the text and a flag to determine if it should be part of the validation set or the training set. If we ignore this flag, we can create a DataBunch containing this data in one line of code:

Contiene una línea por comentario (review), con la etiqueta ('negativo' o 'positivo'), el texto y una bandera para determinar si debe ser parte del conjunto de validación o del conjunto de entrenamiento. Si ignoramos esta bandera, podemos crear un DataBunch que contenga estos datos en una línea de código:

In [ ]:
%timeit
data_lm = TextDataBunch.from_csv(path, 'texts.csv')

By executing this line a process was launched that took a bit of time. Let's dig a bit into it. Images could be fed (almost) directly into a model because they're just a big array of pixel values that are floats between 0 and 1. A text is composed of words, and we can't apply mathematical functions to them directly. We first have to convert them to numbers. This is done in two differents steps: tokenization and numericalization. A `TextDataBunch` does all of that behind the scenes for you.

Al ejecutar esta línea, se lanzó un proceso que tomó un poco de tiempo. Profundicemos un poco en ello. Las imágenes se pueden alimentar (casi) directamente a un modelo porque son solo una gran variedad de valores de píxeles que flotan entre 0 y 1. Un texto está compuesto de palabras, y no podemos aplicarles funciones matemáticas directamente. Primero tenemos que convertirlos a números. Esto se realiza en dos pasos diferentes: tokenización y numeración. Un `TextDataBunch` hace todo eso detrás de escena por usted.

### Tokenization

The first step of processing we make texts go through is to split the raw sentences into words, or more exactly tokens. The easiest way to do this would be to split the string on spaces, but we can be smarter:

- we need to take care of punctuation
- some words are contractions of two different words, like isn't or don't
- we may need to clean some parts of our texts, if there's HTML code for instance

To see what the tokenizer had done behind the scenes, let's have a look at a few texts in a batch.

The texts are truncated at 100 tokens for more readability. We can see that it did more than just split on space and punctuation symbols: 
- the "'s" are grouped together in one token
- the contractions are separated like his: "did", "n't"
- content has been cleaned for any HTML symbol and lower cased
- there are several special tokens (all those that begin by xx), to replace unkown tokens (see below) or to introduce different text fields (here we only have one).

## Tokenización

El primer paso del procesamiento por el que hacemos pasar los textos es dividir las oraciones sin procesar en palabras, o más exactamente tokens. La forma más fácil de hacer esto sería dividir la cadena en espacios, pero podemos ser más inteligentes:

- tenemos que ocuparnos de la puntuación
- algunas palabras son contracciones de dos palabras diferentes, como no es o no
- es posible que necesitemos limpiar algunas partes de nuestros textos, si hay código HTML, por ejemplo

Para ver lo que el tokenizador había hecho detrás de escena, echemos un vistazo a algunos textos en un lote.

Los textos se truncan en 100 tokens para mayor legibilidad. Podemos ver que hizo más que solo dividir en espacio y símbolos de puntuación:
- las "'s" se agrupan en una ficha
- las contracciones se separan como la suya: "did", "n't"
- el contenido se ha limpiado para cualquier símbolo HTML y en minúsculas
- hay varios tokens especiales (todos aquellos que comienzan por xx), para reemplazar tokens desconocidos (ver más abajo) o para introducir diferentes campos de texto (aquí solo tenemos uno).

### Numericalization

Once we have extracted tokens from our texts, we convert to integers by creating a list of all the words used. We only keep the ones that appear at list twice with a maximum vocabulary size of 60,000 (by default) and replace the ones that don't make the cut by the unknown token `UNK`.

The correspondance from ids tokens is stored in the `vocab` attribute of our datasets, in a dictionary called `itos` (for int to string).

In [ ]:
data_lm.vocab.itos[:10]

In [ ]:
data_lm.train_ds[0][0]

In [ ]:
data_lm.train_ds[0][0].data[:10]

## Modelo de Lenguaje (Language model)

In [ ]:
path = untar_data(URLs.IMDB)
path.ls()

Enlace simbolico (symlink / acceso directo) de la carpeta con los datos en la ubicacion correspondiente. 

In [ ]:
#!mkdir ~/.fastai/data
#!mkdir ~/.fastai/data/imdb
#!ln -s /data/home/admin101/.fastai/data/imdb/t* ~/.fastai/data/imdb/
#!ln -s /data/home/admin101/.fastai/data/imdb/u* ~/.fastai/data/imdb/

In [ ]:
#path = Config.data_path()/'imdb'
#path.mkdir(parents=True, exist_ok=True)
#path.ls()

In [ ]:
#(path/'train').ls()

The reviews are in a training and test set following an imagenet structure. The only difference is that there is an `unsup` folder in `train` that contains the unlabelled data.

We're not going to train a model that classifies the reviews from scratch. Like in computer vision, we'll use a model pretrained on a bigger dataset (a cleaned subset of wikipeia called [wikitext-103](https://einstein.ai/research/blog/the-wikitext-long-term-dependency-language-modeling-dataset)). That model has been trained to guess what the next word, its input being all the previous words. It has a recurrent structure and a hidden state that is updated each time it sees a new word. This hidden state thus contains information about the sentence up to that point.

We are going to use that 'knowledge' of the English language to build our classifier, but first, like for computer vision, we need to fine-tune the pretrained model to our particular dataset. Because the English of the reviews left by people on IMDB isn't the same as the English of wikipedia, we'll need to adjust a little bit the parameters of our model. Plus there might be some words extremely common in that dataset that were barely present in wikipedia, and therefore might no be part of the vocabulary the model was trained on.

Los comentarios están en un conjunto de entrenamiento y prueba siguiendo una estructura imagenet. La única diferencia es que hay una carpeta `unsup` en` train` que contiene los datos no etiquetados.

No vamos a entrenar un modelo que clasifique los comentarios desde cero. Al igual que en visión por computadora, utilizaremos un modelo pre-entrenado en un conjunto de datos más grande (un subconjunto limpio de wikipeia llamado [wikitext-103](https://einstein.ai/research/blog/the-wikitext-long-term-dependency -language-modeling-dataset)). Ese modelo ha sido entrenado para adivinar cuál es la siguiente palabra, siendo su entrada todas las palabras anteriores. Tiene una estructura recurrente y un estado oculto que se actualiza cada vez que ve una nueva palabra. Este estado oculto contiene información sobre la oración hasta ese punto.

Vamos a utilizar ese 'conocimiento' del idioma inglés para construir nuestro clasificador, pero primero, como en el caso de la visión por computadora, necesitamos ajustar el modelo previamente entrenado a nuestro conjunto de datos en particular. Debido a que el inglés de las reseñas dejadas por las personas en IMDB no es el mismo que el inglés de wikipedia, necesitaremos ajustar un poco los parámetros de nuestro modelo. Además, puede haber algunas palabras extremadamente comunes en ese conjunto de datos que apenas estaban presentes en wikipedia y, por lo tanto, podrían no ser parte del vocabulario en el que se entrenó el modelo.

### Creating the TextLMDataBunch

This is where the unlabelled data is going to be useful to us, as we can use it to fine-tune our model. Let's create our data object with the data block API (next line takes a few minutes).

### Crear el TextLMDataBunch

Aquí es donde los datos no etiquetados nos serán útiles, ya que podemos usarlos para ajustar nuestro modelo. Creemos nuestro objeto de datos con la API de bloque de datos (la siguiente línea lleva unos minutos).

In [ ]:
path.ls()

In [ ]:
data_lm = (TextList.from_folder(path)
           #Inputs: all the text files in path
            .filter_by_folder(include=['train', 'test', 'unsup']) 
           #We may have other temp folders that contain text files so we only keep what's in train and test
            .split_by_rand_pct(0.1, seed=42)
           #We randomly split and keep 10% (10,000 reviews) for validation
            .label_for_lm()           
           #We want to do a language model so we label accordingly
            .databunch(bs=bs, num_workers=1))

In [ ]:
data_lm.train_ds

In [ ]:
len(data_lm.vocab.itos),len(data_lm.train_ds)

In [ ]:
data_lm

We have to use a special kind of `TextDataBunch` for the language model, that ignores the labels (that's why we put 0 everywhere), will shuffle the texts at each epoch before concatenating them all together (only for training, we don't shuffle for the validation set) and will send batches that read that text in order with targets that are the next word in the sentence.

The line before being a bit long, we want to load quickly the final ids by using the following cell.

Tenemos que usar un tipo especial de `TextDataBunch` para el modelo de lenguaje, que ignora las etiquetas (es por eso que ponemos 0 en todas partes), mezclará los textos en cada época antes de concatenarlos todos juntos (solo para entrenamiento, no lo hacemos), (mezcla aleatoria para el conjunto de validación) y enviará lotes que leen ese texto en orden con los objetivos que son la siguiente palabra en la oración.

La línea antes de ser un poco larga, queremos cargar rápidamente los identificadores finales utilizando la siguiente celda.

In [ ]:
data_lm.show_batch()

Let's save our databunch for next time:

Guardemos nuestros datos para la próxima vez:

In [ ]:
data_lm.save('lm_databunch')

### Loading saved data, and creating the language model

### Cargando datos guardados y creando el modelo de lenguaje

In [ ]:
data_lm = load_data(path, 'lm_databunch', bs=bs)

We can then put this in a learner object very easily with a model loaded with the pretrained weights. They'll be downloaded the first time you'll execute the following line and stored in `~/.fastai/models/` (or elsewhere if you specified different paths in your config file).

Entonces podemos poner esto en un objeto de aprendizaje muy fácilmente con un modelo cargado con los pesos preentrenados. Se descargarán la primera vez que ejecute la siguiente línea y se almacenarán en `~ / .fastai / models /` (o en otro lugar si especificó diferentes rutas en su archivo de configuración).

In [ ]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [ ]:
#wiki_itos = pickle.load(open(Config().model_path()/'wt103-1/itos_wt103.pkl', 'rb')) # dependiendo de la máquina en que se ejecuta 
wiki_itos = pickle.load(open(Config().model_path()/'wt103-fwd/itos_wt103.pkl', 'rb'))

Sitienes un error en el pickle.load ejecuta el bash ajunto

In [ ]:
#%%bash
#jupyter notebook --NotebookApp.iopub_data_rate_limit=1e10
#curl -O http://files.fast.ai/models/wt103/itos_wt103.pkl
#mkdir ../root/.fastai/models/wt103-1
#mv itos_wt103.pkl ../root/.fastai/models/wt103-1/itos_wt103.pkl
#ls /root/.fastai/models/wt103-1 -A

In [ ]:
wiki_itos[:10]

In [ ]:
vocab = data_lm.vocab

In [ ]:
vocab.stoi["stingray"]

In [ ]:
vocab.itos[vocab.stoi["stingray"]]

In [ ]:
vocab.itos[vocab.stoi["mobula"]]

In [ ]:
awd = learn_lm.model[0]

In [ ]:
from scipy.spatial.distance import cosine as dist

In [ ]:
enc = learn_lm.model[0].encoder

In [ ]:
enc.weight.size()

### Difference in vocabulary between IMDB and Wikipedia

We are going to load wiki_itos, which can be downloaded along with wikitext-103.  We will compare the vocabulary from wikitext with the vocabulary in IMDB.  It is to be expected that the two sets have some different vocabulary words, and that is no problem for transfer learning!

In [ ]:
len(wiki_itos)

In [ ]:
len(vocab.itos)

In [ ]:
i, unks = 0, []
while len(unks) < 50:
    if data_lm.vocab.itos[i] not in wiki_itos: unks.append((i,data_lm.vocab.itos[i]))
    i += 1

In [ ]:
wiki_words = set(wiki_itos)

In [ ]:
imdb_words = set(vocab.itos)

In [ ]:
wiki_not_imbdb = wiki_words.difference(imdb_words)

In [ ]:
imdb_not_wiki = imdb_words.difference(wiki_words)

In [ ]:
wiki_not_imdb_list = []

for i in range(100):
    word = wiki_not_imbdb.pop()
    wiki_not_imdb_list.append(word)
    wiki_not_imbdb.add(word)

In [ ]:
wiki_not_imdb_list[:15]

In [ ]:
imdb_not_wiki_list = []

for i in range(100):
    word = imdb_not_wiki.pop()
    imdb_not_wiki_list.append(word)
    imdb_not_wiki.add(word)

In [ ]:
imdb_not_wiki_list[:15]

All words that appear in the IMDB vocab, but not the wikitext-103 vocab, will be initialized to the same random vector in a model.  As the model trains, we will learn these weights.

In [ ]:
vocab.stoi["modernisation"]

In [ ]:
"modernisation" in wiki_words

In [ ]:
vocab.stoi["30-something"]

In [ ]:
"30-something" in wiki_words, "30-something" in imdb_words

In [ ]:
vocab.stoi["linklater"]

In [ ]:
"linklater" in wiki_words, "linklater" in imdb_words

In [ ]:
"house" in wiki_words, "house" in imdb_words

### Generating fake movie reviews (using wiki-text model)

### Generando comentarios falsos de películas (usando el modelo wiki-text)

In [ ]:
TEXT = "The color of the sky is"
N_WORDS = 40
N_SENTENCES = 2

In [ ]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [ ]:
TEXT = "I hated this movie"
N_WORDS = 30
N_SENTENCES = 2

In [ ]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [ ]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

Lowering `temperature` will make the texts less randomized.

Bajar la "temperatura" hará que los textos sean menos random.

In [ ]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.10) for _ in range(N_SENTENCES)))

In [ ]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.10) for _ in range(N_SENTENCES)))

### Training the model

### Entrenando el modelo

Now, we want to choose a good learning rate.

Ahora, queremos elegir una buena tasa de aprendizaje.

In [ ]:
learn_lm.lr_find()

In [ ]:
learn_lm.recorder.plot(skip_end=15)

In [ ]:
lr = 1e-2
lr *= bs/48

In [ ]:
learn_lm.to_fp16();

In [ ]:
learn_lm.fit_one_cycle(1, lr*10, moms=(0.8,0.7))

Since this is relatively slow to train, we will save our weights:

Dado que esto es relativamente lento para entrenar, guardaremos nuestros pesos:

In [ ]:
learn_lm.save('fit_1')

In [ ]:
learn_lm.load('fit_1');

To complete the fine-tuning, we can then unfreeze and launch a new training.

Para completar el ajuste, podemos descongelar y lanzar un nuevo entrenamiento.

In [ ]:
learn_lm.unfreeze()

In [ ]:
learn_lm.fit_one_cycle(10, lr, moms=(0.8,0.7))

In [ ]:
learn_lm.save('fine_tuned')

We have to save not just the model but also it's encoder, the part that's responsible for creating and updating the hidden state. For the next part, we don't care about the part that tries to guess the next word.

Tenemos que guardar no solo el modelo sino también su codificador, la parte responsable de crear y actualizar el estado oculto. Para la siguiente parte, no nos importa la parte que intenta adivinar la siguiente palabra.

In [ ]:
learn_lm.save_encoder('fine_tuned_enc')

### Loading our saved weights

### Cargando nuestros pesos guardados

In [ ]:
learn_lm.load('fine_tuned');

Now that we've trained our model, different representations have been learned for the words that were in IMDB but not wiki (remember that at the beginning we had initialized them all to the same thing):

Ahora que hemos entrenado nuestro modelo, se han aprendido diferentes representaciones para las palabras que estaban en IMDB pero no en wiki (recuerde que al principio las habíamos inicializado todas a la misma cosa):

In [ ]:
enc = learn_lm.model[0].encoder

In [ ]:
np.allclose(enc.weight[vocab.stoi["30-something"], :], 
            enc.weight[vocab.stoi["linklater"], :])

In [ ]:
np.allclose(enc.weight[vocab.stoi["30-something"], :], new_word_vec)

### More generated movie reviews

### Más comentarios de películas generadas

In [ ]:
TEXT = "i liked this movie because"
N_WORDS = 40
N_SENTENCES = 2

In [ ]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [ ]:
TEXT = "This movie was"
N_WORDS = 30
N_SENTENCES = 2

In [ ]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [ ]:
TEXT = "I hated this movie"
N_WORDS = 40
N_SENTENCES = 2

In [ ]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

## Classifier

Now, we'll create a new data object that only grabs the labelled data and keeps those labels. Again, this line takes a bit of time.

Ahora, crearemos un nuevo objeto de datos que solo tome los datos etiquetados y conserve esas etiquetas. Nuevamente, esta línea lleva un poco de tiempo.

In [ ]:
bs=48

In [ ]:
data_clas = (TextList.from_folder(path, vocab=data_lm.vocab)
             #grab all the text files in path
             .split_by_folder(valid='test')
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_folder(classes=['neg', 'pos'])
             #label them all with their folders
             .databunch(bs=bs, num_workers=1))

In [ ]:
data_clas.save('imdb_textlist_class')

In [ ]:
data_clas = load_data(path, 'imdb_textlist_class', bs=bs, num_workers=1)

In [ ]:
data_clas.show_batch()

We can then create a model to classify those reviews and load the encoder we saved before.

Luego podemos crear un modelo para clasificar esas revisiones y cargar el codificador que guardamos antes.

In [ ]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.3) #.to_fp16()
learn_c.load_encoder('fine_tuned_enc')
learn_c.freeze()

In [ ]:
learn_c.lr_find()

In [ ]:
learn_c.recorder.plot()

In [ ]:
learn_c.fit_one_cycle(1, 2e-2, moms=(0.8,0.7))

In [ ]:
learn_c.save('first')

In [ ]:
learn_c.load('first');

In [ ]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [ ]:
learn_c.save('2nd')

In [ ]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

In [ ]:
learn_c.save('3rd')

In [ ]:
learn_c.unfreeze()
learn_c.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

The state of the art for this dataset in 2017 was 94.1%.

El estado del arte para este conjunto de datos en 2017 fue del 94,1%.

In [ ]:
learn_c.save('clas')

In [ ]:
learn_c.predict("I really loved that movie, it was awesome!")

In [ ]:
learn_c.predict("I didn't really love that movie, and I didn't think it was awesome.")